<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [9]:
#Ignoriung Warnings
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [10]:
#importing libs
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

2024-08-06 16:56:06.293347: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
    return angle 
    
def calculate_angle_with_vertical(a, b):
    a = np.array(a)  # First point
    b = np.array(b)  # Mid point (vertex)

    # Calculate the vector from b to a
    vector = a - b

    # Angle of the vector with respect to the vertical axis (y-axis)
    # Vertical axis is along the y-axis, so we use (0, 1) as the vertical reference vector
    vertical_reference = np.array([0, 1])
    
    # Calculate the dot product and magnitudes of the vectors
    dot_product = np.dot(vector, vertical_reference)
    magnitude_vector = np.linalg.norm(vector)
    magnitude_vertical = np.linalg.norm(vertical_reference)

    # Calculate the cosine of the angle
    cos_angle = dot_product / (magnitude_vector * magnitude_vertical)

    # Clamp the value to the range [-1, 1] to avoid numerical issues with arccos
    cos_angle = np.clip(cos_angle, -1.0, 1.0)

    # Calculate the angle in degrees
    angle = np.arccos(cos_angle) * (180.0 / np.pi)
    return angle

In [ ]:
posture=None
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None
message="Good Posture"


## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            v_angle_elbow=calculate_angle_with_vertical(elbow,shoulder)
            v_angle_back=calculate_angle_with_vertical(shoulder,hip)
            
            # Visualize angle
            cv2.putText(image, str(v_angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            



            if angle > 160:
                stage = "down"
            if angle < 30 and stage =='down':
                stage="up"
                counter +=1
                print(counter)

            #posture
            # if (v_angle_elbow>8 or v_angle_hip<170):
            #     if v_angle_back<175 and v_angle_elbow>8:
            #         message="straight back and elbows."
            #     elif v_angle_back<170:
            #         message="straight back.!!"
            #     elif v_angle_elbow>8:
            #         message="straight elbow!!"



                
                
            # else:
            #     message="Good Posture"
            #     # Curl counter logic
            #     if angle > 160:
            #         stage = "down"
            #     if angle < 30 and stage =='down':
            #         stage="up"
            #         counter +=1
            #         print(counter)
                    
                
                
                
                
                
            
                
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (1000,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        # cv2.putText(image, 'STAGE', (65,12), 
        #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        # cv2.putText(image, message, 
        #             (1000,60), 
        #             cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None
message=None

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                   
            # Calculate angles
            angle = calculate_angle(shoulder, elbow, wrist)
            v_angle_elbow = calculate_angle_with_vertical(elbow, shoulder)
            v_angle_back = calculate_angle_with_vertical(shoulder, hip)
            
            # Visualize angle
            cv2.putText(image, str(v_angle_elbow), 
                        tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Curl counter logic
            if angle > 160 and v_angle_elbow < 15 and v_angle_back > 173:
                stage = "down"
                message = 'Good'
            if angle < 30 and stage == 'down' and v_angle_elbow < 15 and v_angle_back > 173:
                stage = "up"
                message = "Good"
                counter += 1
                print(counter)

            if v_angle_elbow > 15 or v_angle_back < 173:
                if v_angle_elbow > 15:
                    message = 'Keep your hands aligned'
                    
                    # Draw circles around the left elbow
                    left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
                    right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
                    cv2.circle(image, (int(left_elbow.x * frame.shape[1]), int(left_elbow.y * frame.shape[0])), 15, (0, 0, 255), -1)                    
                    cv2.circle(image, (int(right_elbow.x * frame.shape[1]), int(right_elbow.y * frame.shape[0])), 15, (0, 0, 255), -1)
                else:
                    message = "Keep your back straight"
                    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    cv2.circle(image, (int(left_hip.x * frame.shape[1]), int(left_hip.y * frame.shape[0])), 15, (0, 0, 255), -1)                    
                    cv2.circle(image, (int(right_hip.x * frame.shape[1]), int(right_hip.y * frame.shape[0])), 15, (0, 0, 255), -1)
                
        except Exception as e:
            print(e)
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        
        # Rep data
        cv2.putText(image, message, (15, 12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (100, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

### Transscript generation

In [13]:
import time

# Video capture
cap = cv2.VideoCapture("right.mp4")

# Curl counter variables
counter = 0 
stage = None
message = None

# Initialize a list to store the transcript data
transcript = []

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    start_time = time.time()
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                   
            # Calculate angles
            angle = calculate_angle(shoulder, elbow, wrist)
            v_angle_elbow = calculate_angle_with_vertical(elbow, shoulder)
            v_angle_back = calculate_angle_with_vertical(shoulder, hip)
            
            # Visualize angle
            cv2.putText(image, str(v_angle_elbow), 
                        tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Curl counter logic
            if angle > 160 and v_angle_elbow < 15 and v_angle_back > 173:
                stage = "down"
                message = 'Good'
            if angle < 30 and stage == 'down' and v_angle_elbow < 15 and v_angle_back > 173:
                stage = "up"
                message = "Good"
                counter += 1
                print(counter)
                # Log the time and rep count
                current_time = time.time() - start_time
                transcript.append(f"Time: {current_time:.2f} sec, Rep: {counter}")
                message += f" | Time: {current_time:.2f} sec"
            
            if v_angle_elbow > 15 or v_angle_back < 173:
                if v_angle_elbow > 15:
                    message = 'Keep your hands aligned'
                    
                    # Draw circles around the left elbow
                    left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
                    cv2.circle(image, (int(left_elbow.x * frame.shape[1]), int(left_elbow.y * frame.shape[0])), 15, (0, 0, 255), -1)
                    
                    # Optionally, if you want to highlight the right elbow as well:
                    right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
                    cv2.circle(image, (int(right_elbow.x * frame.shape[1]), int(right_elbow.y * frame.shape[0])), 15, (0, 0, 255), -1)
                else:
                    message = "Keep your back straight"
                
        except Exception as e:
            print(e)
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        
        # Rep data
        cv2.putText(image, message, (15, 12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (100, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Write the transcript to a file
    with open('exercise_transcript.txt', 'w') as f:
        for entry in transcript:
            f.write(entry + '\n')


In [ ]:
transcript

### Using movieyou.editor 

In [ ]:
!pip install moviepy

In [ ]:
pip install ez_setup

In [1]:
from moviepy.editor import *

video = VideoFileClip("right.mp4")

# Make the text. Many more options are available.
txt_clip = ( TextClip("My Holidays 2013",fontsize=70,color='white')
             .set_position('center')
             .set_duration(10) )

result = CompositeVideoClip([video, txt_clip]) # Overlay text on video
# result.write_videofile("myHolidays_edited.webm",fps=25) # Many options...
result

In [14]:
result.write_videofile("myHolidays_edited.webm",fps=30) # Many options...

Moviepy - Building video myHolidays_edited.webm.
MoviePy - Writing audio in myHolidays_editedTEMP_MPY_wvf_snd.ogg


MoviePy - Done.
Moviepy - Writing video myHolidays_edited.webm



Moviepy - Done !
Moviepy - video ready myHolidays_edited.webm


In [8]:
from IPython.display import HTML

HTML("""
    <video alt="test" controls style="width:400px;">
        <source src="myHolidays_edited.webm" type="video/mp4" >
    </video>
""")

In [ ]:
import pygame

In [ ]:
!pip install pygame

In [ ]:
pip install moviepy

In [ ]:
!brew install imagemagick


In [ ]:
!pip install pygame